In [11]:
import plotly.graph_objects as go
import pandas as pd

data= pd.read_csv("ICBC_reported_crashes_Full_Data_data.csv", dtype={
    "Crash Breakdown 2": "string",
    "Date Of Loss Year": "string",
    "Animal Flag": "string",
    "Crash Severity": "string",
    "Cyclist Flag": "string",
    "Day Of Week": "string",
    "Derived Crash Congifuration" : "string",	
    "Intersection Crash": "string",
    "Month Of Year": "string",
    "Motorcycle Flag": "string",
    "Municipality Name (ifnull)": "string",
    "Parking Lot Flag": "string",
    "Pedestrian Flag": "string",
    "Region": "string",
    "Street Full Name (ifnull)": "string",
    "Time Category": "string",
    "Municipality Name": "string",
    "Road Location Description": "string",
    "Street Full Name": "string",
    "Total Crashes": "string"

})
#print(data.head())
data = data.head()
print(data['Month Of Year'][0])
#data = data.head().to_dict()

fig = go.Figure(data=go.Scatter(x=data['Street Full Name'],
                                y=data['Month Of Year'],
                                mode='markers',
                                marker_color='red'))
fig.update_layout(title='Population of USA States')
fig.show()



/home/tia/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.



APRIL
